In [1]:
%matplotlib inline

from ROOT import TFile,vector,TGraph
import ROOT
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
from numpy import mean
from math import sqrt,acos,cos,sin,pi,exp,log,isnan,atan2
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from numpy import asarray
from root_pandas import read_root
from matplotlib import gridspec
from scipy import stats

Welcome to JupyROOT 6.12/06


/usr/local/lib/python2.7/dist-packages/root_numpy/__init__.py:46: RuntimeWarning: numpy 1.16.3 is currently installed but you installed root_numpy against numpy 1.14.3. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)


In [2]:
def bless_mcinfo(row):
    try:
        idx = tuple((row['run'],row['subrun'],row['event']))
        if MCInfo[idx]['nuPDG'] in [12,-12]:
            return 0
        else:
            return 1
    except:
        return 0

def bless_scedr(row):
    scedr = float(-1)
    dx = row['Xreco'] - row['parentSCEX']
    dy = row['Yreco'] - row['parentSCEY']
    dz = row['Zreco'] - row['parentSCEZ']
    dr2 = dx*dx+dy*dy+dz*dz
    dr = np.sqrt(dr2)

    scedr = dr
    return scedr

def bless_ccqe_reweight_numu(row):
    rw = float(1)
    if row['interactionType']==1001:
        rw = xsec_tune1_graph_numu.Eval(row['energyInit']) / xsec_mcc9_graph_numu.Eval(row['energyInit'])
    return rw

def bless_ccqe_reweight_nue(row):
    rw = float(1)
    if row['interactionType']==1001:
        rw = xsec_tune1_graph_nue.Eval(row['energyInit']) / xsec_mcc9_graph_nue.Eval(row['energyInit'])
    return rw


In [3]:
# Time to load the good runs list
good_df = pd.read_csv('../data/goodruns_intersec_all_runs.txt')
good_df['good'] = 1

# loat pmt precuts
df_pmtprecut_bnb = read_root('../data/mcc9_v13_bnb_overlay/pmt_precut_dump.root')
df_pmtprecut_nue = read_root('../data/mcc9_v13_nueintrinsic_overlay/pmt_precut_dump.root')
df_pmtprecut_ext = read_root('../data/mcc9jan_extbnb/pmt_precut_dump.root')
df_pmtprecut_data = read_root('../data/mcc9jan_bnb5e19/pmt_precut_dump.root')

RSE=['run','subrun','event']

In [4]:
df_bnb = read_root('../data/mcc9_v13_bnb_overlay/FinalVertexVariables-prime_vOct31_FULL.root')
df_bnb_truth = read_root('../data/mcc9_v13_bnb_overlay/MCInfo.root','EventMCINFO_DL')

# merge dfs
df_bnb = pd.merge(df_bnb,df_bnb_truth,on=RSE)
df_bnb = pd.merge(df_bnb,good_df,on='run')
df_bnb = pd.merge(df_bnb,df_pmtprecut_bnb,on=RSE)

df_numu = df_bnb.query('parentPDG==14 or parentPDG==-14')
df_numu_goodruns = df_numu.query("good==1")
df_numu_goodruns_pmtprecut = df_numu_goodruns.query("passed==1")

# restrict to 5e19 run range
#df_numu_goodruns_pmtprecut = df_numu_goodruns_pmtprecut.query('run>=5119 and run<=5955')

In [5]:
df_nue = read_root('../data/mcc9_v13_nueintrinsic_overlay/FinalVertexVariables-prime_vOct31_FULL.root')
df_nue_truth = read_root('../data/mcc9_v13_nueintrinsic_overlay/mc_info_mcc9_v13_nueintrinsics_overlay_run1.root','EventMCINFO_DL')

df_nue = pd.merge(df_nue,df_nue_truth,on=RSE)
df_nue = pd.merge(df_nue,good_df,on='run')
df_nue = pd.merge(df_nue,df_pmtprecut_nue,on=RSE)

df_nue_goodruns = df_nue.query("good==1")
df_nue_goodruns_pmtprecut = df_nue_goodruns.query("passed==1")

# restrict to 5e19 run range
#df_nue_goodruns_pmtprecut = df_nue_goodruns_pmtprecut.query('run>=5119 and run<=5955')

In [6]:
df_ext = read_root('../data/mcc9jan_extbnb/FinalVertexVariables-prime_vOct31_FULL.root')

df_ext = pd.merge(df_ext,good_df,on='run')
df_ext = pd.merge(df_ext,df_pmtprecut_ext,on=RSE)

df_ext_goodruns = df_ext.query("good==1")
df_ext_goodruns_pmtprecut = df_ext_goodruns.query("passed==1")

# restrict to 5e19 run range
#df_ext_goodruns_pmtprecut = df_ext_goodruns_pmtprecut.query('run>=5119 and run<=5955')

In [7]:
df_data = read_root('../data/mcc9jan_bnb5e19/FinalVertexVariables-prime_vOct31_FULL.root')

df_data = pd.merge(df_data,good_df,on='run')
df_data = pd.merge(df_data,df_pmtprecut_data,on=RSE)

df_data_goodruns = df_data.query("good==1")
df_data_goodruns_pmtprecut = df_data_goodruns.query("passed==1")

In [8]:
df_numu_goodruns_pmtprecut.insert(0,'scedir_corrected',df_numu_goodruns_pmtprecut.apply(bless_scedr,axis=1))

In [9]:
df_nue_goodruns_pmtprecut.insert(0,'scedir_corrected',df_nue_goodruns_pmtprecut.apply(bless_scedr,axis=1))

In [10]:
xsec_mcc9_file = ROOT.TFile('../data/xsec_graphs_mcc9_broken.root')
xsec_mcc9_graph_numu = xsec_mcc9_file.Get('nu_mu_Ar40/qel_cc_n')
xsec_mcc9_graph_nue = xsec_mcc9_file.Get('nu_e_Ar40/qel_cc_n')
xsec_tune1_file = ROOT.TFile('../data/xsec_graphs_tune1.root')
xsec_tune1_graph_numu = xsec_tune1_file.Get('nu_mu_Ar40/qel_cc_n')
xsec_tune1_graph_nue = xsec_tune1_file.Get('nu_e_Ar40/qel_cc_n')

In [11]:
df_numu_goodruns_pmtprecut.insert(0,'geniereweight',df_numu_goodruns_pmtprecut.apply(bless_ccqe_reweight_numu,axis=1))

In [12]:
df_nue_goodruns_pmtprecut.insert(0,'geniereweight',df_nue_goodruns_pmtprecut.apply(bless_ccqe_reweight_nue,axis=1))

In [13]:
tag = 'Oct31_fullrange'
df_data_goodruns_pmtprecut.to_pickle('../data/pickles/data_goodruns_pmtprecut_%s.pkl'%tag)
df_numu_goodruns_pmtprecut.to_pickle('../data/pickles/numu_goodruns_pmtprecut_%s.pkl'%tag)
df_nue_goodruns_pmtprecut.to_pickle('../data/pickles/nue_goodruns_pmtprecut_%s.pkl'%tag)
df_ext_goodruns_pmtprecut.to_pickle('../data/pickles/ext_goodruns_pmtprecut_%s.pkl'%tag)

# Get POT for mc

In [ ]:
df_pot = read_root('../data/mcc9_v13_nueintrinsic_overlay/pot_scrape.root','pot_tree',columns=['run', 'subrun','pot'])
RS=['run','subrun']

df_pot = pd.merge(df_nue_goodruns_pmtprecut.query('run>=5119 and run<=5955'),df_pot,on=RS)
df_pot.drop_duplicates(RS,inplace=True)
print df_pot['pot'].values.sum()

In [ ]:
# Get list of RS
RS=['run','subrun']
df_rs = df_ext_goodruns.drop_duplicates(RS)

f = open('RS_mcc9_ext_goodruns.txt','w')
for e in xrange(0,len(df_rs)):
    f.write('%i %i\n' % (df_rs['run'].values[e], df_rs['subrun'].values[e]))
f.close()    

print len(df_rs),'%i %i\n' % (df_rs['run'].values[-1], df_rs['subrun'].values[-1])